# Map Reduce
## Weighted mean and Variance
### Exercise 1
Write functions to compute the average value and variance using for loops

In [1]:
def weightedLoop(X, P):
    mean = 0
    term = 0
    for x,p in zip(X,P):
        mean += p*x
        term += p*x**2
    var = term - mean**2
    return mean, var

In [2]:
X = [5, 1, 2, 3, 1, 2, 5, 4]
P = [0.05, 0.05, 0.15, 0.05, 0.15, 0.2, 0.1, 0.25]
weightedLoop(X,P)

(2.8, 1.9600000000000017)

### Exercise 2
Write functions to compute the average value and variance using `map` and `reduce`

In [3]:
from functools import reduce

In [4]:
def weightedMapReduce(X, P):
    mult = lambda a,b: a*b
    add = lambda a,b: a+b
    
    mean = reduce(add, (map(mult, P, X)))
    var = reduce(add, (map(mult, P, map(lambda x:x**2, X)))) - mean**2
    return mean, var

In [5]:
weightedMapReduce(X,P)

(2.8, 1.9600000000000017)

## Wordcount
## Map - Read file and return a key/value pairs
### Exercise 3
Write a function `mapper` with a single file name as input that returns a sorted sequence of tuples (word, 1) values.

In [6]:
def mapper(path):
    with open(path, 'r') as f:
        return [(word, 1)
                for word in sorted(f.read().replace(".", " ").lower().split())]

In [7]:
# Regenerate file
from lorem import text
with open("sample.txt", "w") as f:
    for i in range(100):
        f.write(text())
        
mapper("sample.txt")[:5]

[('adipisci', 1),
 ('adipisci', 1),
 ('adipisci', 1),
 ('adipisci', 1),
 ('adipisci', 1)]

## Partition
### Exercise 4
Create a function named `partitioner` that stores the key/value pairs from mapper that group (word, 1) pairs into a list as:

In [8]:
def partitioner(pairs):
    partition = dict()
    for key, val in pairs:
        partition[key] = partition.get(key, []) + [val]
    return partition

In [9]:
p = partitioner(mapper("sample.txt"))